# <center>Spotify Pipelining</center>

In [14]:
#ty
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_column', None)

In [58]:
# ty

# spliting the dataset into test and split                                   done
# separate the categorical and numerical features                            done
# Data cleaning the dataset, some columns are not in the proper format       done
# Data preprocessing for caegorical and numerical data                        done
# feature engineering                                                         done
# combining the Data preprocessing process for both                           done

In [6]:
# splittying the dataset into test and split

df = pd.read_csv('278k_song_labelled.csv', index_col='Unnamed: 0')
X = df.drop('labels', axis=1)
y = df['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [11]:

# Identify numerical and categorical columns
num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns

In [8]:
numerical_cols

Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object')

In [36]:
# feaure engineering class

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return self.feature_eng(X)

    @staticmethod
    def feature_eng(df):
        # Addition
        df['ET'] = df['energy'] + df['tempo']
        df['VD'] = df['valence'] + df['danceability']
        # Multiplication
        df['IA'] = df['instrumentalness'] * df['acousticness']
        # Average
        df['ETV'] = (df['energy'] + df['tempo'] + df['valence']) / 3
        # Weighted average
        df['DI'] = df['danceability'] * 0.1 + df['instrumentalness'] * 0.2
        # Concatenating all the features used
        df['ETVDIA'] = df['energy'] + df['tempo'] + df['valence'] + df['danceability'] + df['instrumentalness'] * df['acousticness']
        
        return df

In [37]:
# Define preprocessing for feature engineering
numerical_transformer_feature_engineering = Pipeline(steps=[
    ('feature_engineering', FeatureEngineering()),
    ('scaler', RobustScaler())
])



In [38]:
# Define preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler())
])

In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('feat_enginering', numerical_transformer_feature_engineering, X_train.columns)
    ])

In [40]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', RobustScaler())]),
                                 Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object')),
                                ('feat_enginering',
                                 Pipeline(steps=[('feature_engineering',
                                                  FeatureEngineering()),
                                                 ('scaler', RobustScaler())]),
                                 Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object'))])

In [41]:
# Define the model
model = XGBClassifier()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

In [42]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object')),
                                                 ('feat_enginering',
                                                  Pipeline(steps=[('featur...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [43]:
# Train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object')),
                                                 ('feat_enginering',
                                                  Pipeline(steps=[('featur...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [44]:
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

In [49]:
y_pred_test

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [53]:
#Evaluate the model
print("Training Data Evaluation:")
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print("Accuracy:", accuracy_score(y_test, y_pred_test))


Training Data Evaluation:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     16412
           1       0.96      0.97      0.96     21286
           2       0.95      0.94      0.95      9413
           3       0.98      0.99      0.98      8477

    accuracy                           0.96     55588
   macro avg       0.97      0.96      0.97     55588
weighted avg       0.96      0.96      0.96     55588

[[15898   360    56    98]
 [  314 20541   395    36]
 [   57   497  8851     8]
 [   87    36     4  8350]]
Accuracy: 0.9649564654241922


### Saving our model as a file

In [59]:
#imporing he dill library
import dill

In [62]:
# Save the pipeline
dill.dump(pipeline, open('spotify_classification_pipeline_dill', 'wb'))
spotify_model = dill.load(open('spotify_classification_pipeline_dill','rb'))

In [63]:
spotify_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['duration (ms)', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'spec_rate'],
      dtype='object')),
                                                 ('feat_enginering',
                                                  Pipeline(steps=[('featur...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [64]:
type(spotify_model)

sklearn.pipeline.Pipeline

In [65]:
spotify_model.predict(X_test)

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)